<a href="https://colab.research.google.com/github/Ilona-11/UAS-DeepLearning/blob/main/UAS_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Maria Ilona Junide Bria (202211420006)**

# **Classification Gambar dan Bahasa**

### **Klasifikasi Hewan Kucing Dan Anjing Menggunakan Convolutional Neural Network (CNN) DenseNet**

DenseNet (Densely Connected Convolutional Network) adalah arsitektur convolusional yang unik karena setiap layer tidak hanya menerima input dari layer sebelumnya secara langsung, tapi menerima dan mengkonsolidasikan output (feature maps) dari semua layer sebelumnya. Dengan kata lain, untuk setiap layer l, inputnya adalah hasil concatenation dari output semua layer sebelumnya [x0,x1,...,xl−1].


# **Import dan Download Dataset**

Menggunakan library kagglehub untuk mengunduh dataset Cats and Dogs dari Kaggle. Dataset ini berisi folder untuk training dan testing dengan gambar kucing dan anjing. File dan folder dicek dan disiapkan untuk proses selanjutnya.

In [1]:
import kagglehub

# Download dataset dari Kaggle
path = kagglehub.dataset_download("tongpython/cat-and-dog")

print("Path to dataset files:", path)

!ls /kaggle/input

import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
print("done")


Using Colab cache for faster access to the 'cat-and-dog' dataset.
Path to dataset files: /kaggle/input/cat-and-dog
cat-and-dog
done


# **Import Library CNN (DenseNet)**

Import library TensorFlow dan Keras serta modul DenseNet201 pretrained dari ImageNet, yang akan digunakan sebagai backbone model.

*   Tensorflow sebagai Kerangka utama deep learning.
*   ImageDataGenerator dari Keras untuk augmentasi dan preprocessing gambar.
*   DenseNet201 sebagai arsitektur backbone pretrained dari ImageNet yang akan diterapkan untuk transfer learning.
*   Module seperti Model, Dense, Dropout, GlobalAveragePooling2D, dan Adam adalah komponen penting untuk membangun, memodifikasi, dan mengoptimalkan arsitektur jaringan yang akan digunakan pada tugas klasifikasi dua kelas (biner)


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


# **Siapkan Generator Gambar**

Menggunakan ImageDataGenerator untuk augmentasi data training seperti rotasi, translasi, zoom, flip horizontal, dan sekaligus membagi data menjadi training dan validation dengan validation_split=0.2. Ukuran input yang dipakai adalah 224x224 piksel sesuai DenseNet.

In [3]:
IMG_SIZE = (224, 224)  # ukuran sesuai dengan DenseNet
BATCH_SIZE = 10

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/cat-and-dog/training_set/training_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/cat-and-dog/training_set/training_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    "/kaggle/input/cat-and-dog/test_set/test_set",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


# **Bangun Model DenseNet**

Model DenseNet201 digunakan tanpa top layer (fully connected) bawaan. Layer awal dibekukan kecuali 20 layer terakhir yang di-unfreeze untuk fine tuning. Kemudian ditambahkan global average pooling, dense layer 512 neuron, dropout 0.5 untuk regularisasi, dan output layer sigmoid untuk klasifikasi binary.


In [4]:
# Menggunakan DenseNet201 (pretrained dari ImageNet)
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tuning sebagian layer
num_unfreeze_layers = 20
for layer in base_model.layers[:-num_unfreeze_layers]:
    layer.trainable = False
for layer in base_model.layers[-num_unfreeze_layers:]:
    layer.trainable = True

# Tambahkan lapisan klasifikasi
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 19,306,049 (73.65 MB)

 Trainable params: 1,585,985 (6.05 MB)

 Non-trainable params: 17,720,064 (67.60 MB)